In [ ]:
# GRANT ACCESS TO ALL LISTS IN A SITE
import time
import requests
import os
from selenium import webdriver
from selenium.webdriver.edge.service import Service

# --- CONFIGURATION ---
SITE_URL = "https://prometeontyregroup-my.sharepoint.com/personal/mazen_ahmed_st_prometeon_com"
USER_EMAIL = "nada.mohamed.st@prometeon.com"

# Lists to IGNORE
# Lists to IGNORE
EXCLUDED_LISTS = [
    "Documents", "Form Templates", "Style Library", 
    "Site Assets", "Site Pages", "Maintenance Mode",
    "Social", "MicroFeed", "TaxonomyHiddenList",
    # --- NEW EXCLUSIONS ---
    "Documenti",
    "Pagine del sito",
    "Social networking"
]

def get_sharepoint_cookies(site_url):
    print(">>> LAUNCHING EDGE (Offline Mode)...")
    
    # 1. SETUP DRIVER MANUALLY
    # We look for msedgedriver.exe in the CURRENT folder
    driver_path = "msedgedriver.exe"
    
    if not os.path.exists(driver_path):
        print(f"ERROR: Could not find '{driver_path}' in this folder!")
        print("Please download it from: https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/")
        print("And place it next to this script.")
        return None

    try:
        service = Service(executable_path=driver_path)
        driver = webdriver.Edge(service=service)
    except Exception as e:
        print(f"Error launching Edge: {e}")
        return None
    
    # 2. Go to the site
    driver.get(site_url)
    
    # 3. WAIT FOR USER LOGIN
    print(">>> -------------------------------------------------------------")
    print(">>> PLEASE LOG IN IN THE EDGE WINDOW.")
    print(">>> Do your 2FA (Phone/SMS).")
    print(">>> Once you see your files/folders, come back here and PRESS ENTER.")
    print(">>> -------------------------------------------------------------")
    input("Press Enter to continue...") 
    
    selenium_cookies = driver.get_cookies()
    driver.quit()
    
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
        
    print(">>> Login successful! Cookies captured.")
    return session

def get_request_digest(session, site_url):
    url = f"{site_url}/_api/contextinfo"
    headers = {
        "Accept": "application/json;odata=verbose",
        "Content-Type": "application/json;odata=verbose"
    }
    response = session.post(url, headers=headers)
    if response.status_code == 200:
        return response.json()['d']['GetContextWebInformation']['FormDigestValue']
    return None

def main():
    # 1. Login
    session = get_sharepoint_cookies(SITE_URL)
    if not session: return
    
    # 2. Get Token
    digest = get_request_digest(session, SITE_URL)
    if not digest:
        print("Failed to get security token. Exiting.")
        return

    headers = {
        "Accept": "application/json;odata=verbose",
        "Content-Type": "application/json;odata=verbose",
        "X-RequestDigest": digest
    }

    # 3. Get User ID
    print(f"\n>>> Resolving User ID for {USER_EMAIL}...")
    user_url = f"{SITE_URL}/_api/web/siteusers/getByEmail('{USER_EMAIL}')"
    r_user = session.get(user_url, headers=headers)
    
    if r_user.status_code != 200:
        print("User not found in site directory. Attempting to add...")
        ensure_url = f"{SITE_URL}/_api/web/ensureUser('{USER_EMAIL}')"
        r_user = session.post(ensure_url, headers=headers)
        if r_user.status_code != 200:
            print("Could not find user. Check email.")
            return

    user_id = r_user.json()['d']['Id']
    print(f"User ID found: {user_id}")

    # 4. Get Role ID (Contribute)
    role_url = f"{SITE_URL}/_api/web/roledefinitions/getbyname('Contribute')"
    r_role = session.get(role_url, headers=headers)
    role_id = r_role.json()['d']['Id'] if r_role.status_code == 200 else 1073741827

    # 5. Loop Lists
    print("\n>>> Fetching Lists...")
    lists_url = f"{SITE_URL}/_api/web/lists"
    r_lists = session.get(lists_url, headers=headers)
    all_lists = r_lists.json()['d']['results']

    i=1
    for lst in all_lists:
        title = lst['Title']
        if lst['Hidden'] or title in EXCLUDED_LISTS:
            continue

        print(f"{i} - Giving access to {USER_EMAIL} on: {title}...", end=" ")
        i += 1

        # Break Inheritance
        break_url = f"{SITE_URL}/_api/web/lists/getbytitle('{title}')/breakroleinheritance(copyRoleAssignments=true, clearSubscopes=true)"
        session.post(break_url, headers=headers)

        # Add User
        add_role_url = f"{SITE_URL}/_api/web/lists/getbytitle('{title}')/roleassignments/addroleassignment(principalid={user_id}, roledefid={role_id})"
        r_add = session.post(add_role_url, headers=headers)

        if r_add.status_code == 200:
            print("[SUCCESS]")
        else:
            print(f"[ERROR] {r_add.status_code}")

    print("\n>>> All done!")
    # input("Press Enter to exit.")

if __name__ == "__main__":
    main()

>>> LAUNCHING EDGE (Offline Mode)...
>>> -------------------------------------------------------------
>>> PLEASE LOG IN IN THE EDGE WINDOW.
>>> Do your 2FA (Phone/SMS).
>>> Once you see your files/folders, come back here and PRESS ENTER.
>>> -------------------------------------------------------------
>>> Login successful! Cookies captured.

>>> Resolving User ID for nada.mohamed.st@prometeon.com...
User ID found: 5

>>> Fetching Lists...
1 - Processing: Building task instructions... [SUCCESS]
2 - Processing: Building_Electrical_PM_2026... [SUCCESS]
3 - Processing: Building_Mechanical_PM_2026... [SUCCESS]
4 - Processing: BY1_Electrical_PM_2026... [SUCCESS]
5 - Processing: BY1_Mechanical_PM_2026... [SUCCESS]
6 - Processing: BY2_Electrical_PM_2026... [SUCCESS]
7 - Processing: BY2_Mechanical_PM_2026... [SUCCESS]
8 - Processing: BY3_Electrical_PM_2026... [SUCCESS]
9 - Processing: BY3_Mechanical_PM_2026... [SUCCESS]
10 - Processing: BY4_Electrical_PM_2026... [SUCCESS]
11 - Processing: BY4

In [ ]:
import time
import requests
import os
from selenium import webdriver
from selenium.webdriver.edge.service import Service

# --- CONFIGURATION ---
SITE_URL = "https://prometeontyregroup-my.sharepoint.com/personal/mazen_ahmed_st_prometeon_com"
USER_EMAIL = "nada.mohamed.st@prometeon.com" # The user to remove

# Lists to IGNORE
EXCLUDED_LISTS = [
    "Documents", "Form Templates", "Style Library", 
    "Site Assets", "Site Pages", "Maintenance Mode",
    "Social", "MicroFeed", "TaxonomyHiddenList",
    # --- NEW EXCLUSIONS ---
    "Documenti",
    "Pagine del sito",
    "Social networking"
]

def get_sharepoint_cookies(site_url):
    print(">>> LAUNCHING EDGE...")
    driver_path = "msedgedriver.exe"
    if not os.path.exists(driver_path): return None

    try:
        service = Service(executable_path=driver_path)
        driver = webdriver.Edge(service=service)
    except: return None
    
    driver.get(site_url)
    print(">>> LOG IN and press ENTER here when ready.")
    input("Press Enter to continue...") 
    
    selenium_cookies = driver.get_cookies()
    driver.quit()
    
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'], domain=cookie['domain'])
    return session

def get_request_digest(session, site_url):
    url = f"{site_url}/_api/contextinfo"
    headers = {"Accept": "application/json;odata=verbose"}
    response = session.post(url, headers=headers)
    return response.json()['d']['GetContextWebInformation']['FormDigestValue'] if response.status_code == 200 else None

def main():
    session = get_sharepoint_cookies(SITE_URL)
    digest = get_request_digest(session, SITE_URL)
    
    headers = {
        "Accept": "application/json;odata=verbose",
        "Content-Type": "application/json;odata=verbose",
        "X-RequestDigest": digest
    }

    # 1. FIND NADA'S ID
    print(f"\n>>> Finding User ID for {USER_EMAIL}...")
    user_url = f"{SITE_URL}/_api/web/siteusers/getByEmail('{USER_EMAIL}')"
    r_user = session.get(user_url, headers=headers)
    
    if r_user.status_code != 200:
        print("User not found on site. Nothing to remove.")
        return

    user_id = r_user.json()['d']['Id']
    print(f"User ID is: {user_id}")

    # 2. REMOVE HER FROM LISTS
    print("\n>>> Removing User from Lists...")
    lists_url = f"{SITE_URL}/_api/web/lists"
    r_lists = session.get(lists_url, headers=headers)
    all_lists = r_lists.json()['d']['results']

    i=1
    for lst in all_lists:
        title = lst['Title']
        if lst['Hidden'] or title in EXCLUDED_LISTS: continue

        print(f"{i} - Removing {USER_EMAIL} from: {title}...", end=" ")
        i += 1

        # COMMAND: RemoveRoleAssignment (Deletes specific user ID from permissions)
        remove_url = f"{SITE_URL}/_api/web/lists/getbytitle('{title}')/roleassignments/removeroleassignment(principalid={user_id})"
        
        r_remove = session.post(remove_url, headers=headers)

        if r_remove.status_code == 200:
            print("[REMOVED]")
        elif r_remove.status_code == 404:
            print("[WAS NOT THERE]") # She wasn't on the list anyway
        else:
            print(f"[ERROR {r_remove.status_code}]")

    print("\n>>> Done.")
    # input("Press Enter to exit.")

if __name__ == "__main__":
    main()

>>> LAUNCHING EDGE...
>>> LOG IN and press ENTER here when ready.

>>> Finding User ID for nada.mohamed.st@prometeon.com...
User ID is: 5

>>> Removing User from Lists...
1 - Removing from: Building task instructions... [REMOVED]
2 - Removing from: Building_Electrical_PM_2026... [REMOVED]
3 - Removing from: Building_Mechanical_PM_2026... [REMOVED]
4 - Removing from: BY1_Electrical_PM_2026... [REMOVED]
5 - Removing from: BY1_Mechanical_PM_2026... [REMOVED]
6 - Removing from: BY2_Electrical_PM_2026... [REMOVED]
7 - Removing from: BY2_Mechanical_PM_2026... [REMOVED]
8 - Removing from: BY3_Electrical_PM_2026... [REMOVED]
9 - Removing from: BY3_Mechanical_PM_2026... [REMOVED]
10 - Removing from: BY4_Electrical_PM_2026... [REMOVED]
11 - Removing from: BY4_Mechanical_PM_2026... [REMOVED]
12 - Removing from: BY5_Electrical_PM_2026... [REMOVED]
13 - Removing from: BY5_Mechanical_PM_2026... [REMOVED]
14 - Removing from: BY6_Electrical_PM_2026... [REMOVED]
15 - Removing from: BY6_Mechanical_PM_20